# Homework 9:

Repeat the procedure we did in class to map the cities of the US using a map and the results of a PCoA, but using a different country of your choosing.

Tips:

These are the packages that you will need:

```
import geopy.distance
import pandas as pd
import plotly.express as px
import skbio
```

You will also need the `world_cities.csv` file to get the city names and coordinates for your country.

1. Choose your country.
2. Get only the cities of that country from the `world_cities.csv` file.
3. Sort the cities by population and select the 20 to 50 most populated cities.
4. Create a data frame with only these cities.
5. Calculate the distance between the cities using their latitude/longitude data from your data frame (geopy).
6. Create a data frame with a table of all the distances between each paur of cities. This needs to be in a distance matrix format, just like we imported from the `cities.csv` file for the US.
7. Perform the PCoA using skbio.
8. Plot the results of the PCoA using plotly express.
9. Plot the map of the country with the cities using the latitude/longitude from your table using plotly express.
10. That's it!

In [16]:
import geopy.distance
from geopy.distance import lonlat, distance

import pandas as pd
import plotly.express as px
import pcoa

all_city = pd.read_csv('world_cities.csv', delimiter='\t')
japan = all_city.loc[all_city['country.etc'] == 'Japan']

japan = japan.sort_values('pop', ascending=False)
japan_top = japan.iloc[:20]

def dist (lonlat1, lonlat2):
    return geopy.distance.distance(lonlat(*lonlat1), lonlat(*lonlat2)).miles

dist_matrix = pd.DataFrame( japan_top, columns=japan_top['name'].unique(), index=japan_top['name'].unique())

for i in dist_matrix.columns:
    for j in dist_matrix.index:
        lonlat1 = japan_top.loc[japan_top['name'] == i, ['long', 'lat']].values[0]
        lonlat2 = japan_top.loc[japan_top['name'] == j, ['long', 'lat']].values[0]
        dist_matrix.loc[j, i] = dist(lonlat1, lonlat2)

dist_matrix
my_pcoa = pcoa.pcoa(dist_matrix)

my_pcoa['loadings']


px.scatter(my_pcoa['loadings'], x='PC1', y='PC2', text=my_pcoa['loadings'].index)

In [23]:
fig2 = px.scatter_geo(japan_top, lon = 'long', lat = 'lat',text="name", size="pop") 
fig2.update_geos(
    projection=dict(type='natural earth'),
    center=dict(lon=138, lat=36),
    lataxis_range=[30, 50],
    lonaxis_range=[130, 150]
)